In [1]:
pip install geopy

You should consider upgrading via the 'C:\Users\Иван\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import sklearn
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
from geopy.distance import great_circle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, mean_squared_error
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# **Датасет (Feature Engineering)**

In [7]:
test = pd.read_csv('C:\\Users\\Иван\\Downloads\\test_2.csv')
train = pd.read_csv('C:\\Users\\Иван\\Downloads\\train_2.csv')

In [8]:
train.head()

,Id,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,TARGET
0,57094,1.0,3,0,2016-05-19 18:06:27,A FABULOUS 3BR IN MIDTOWN WEST! PERFECT APAR...,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"['Laundry In Unit', 'No Fee', 'Elevator']",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium
1,33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,Renovated Kitchen and Bathroom!,55 River Drive South,"['Dogs Allowed', 'Cats Allowed', 'No Fee']",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium
2,60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,RARE AND BEST DEAL ON THE MARKET!!!! PERFECT S...,W 77 Street,"['Elevator', 'Hardwood Floors']",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low
3,53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,Newly renovated flex 2 apartment offers the ne...,John Street,"['Swimming Pool', 'Doorman', 'Elevator', 'Fitn...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low
4,592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,LOW FEE apartments do not come around like thi...,West 16th Street,"['Laundry in Building', 'Laundry in Unit', 'Di...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34546 entries, 0 to 34545
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               34546 non-null  int64  
 1   bathrooms        34546 non-null  float64
 2   bedrooms         34546 non-null  int64  
 3   building_id      34546 non-null  object 
 4   created          34546 non-null  object 
 5   description      33509 non-null  object 
 6   display_address  34458 non-null  object 
 7   features         34546 non-null  object 
 8   latitude         34546 non-null  float64
 9   listing_id       34546 non-null  int64  
 10  longitude        34546 non-null  float64
 11  manager_id       34546 non-null  object 
 12  photos           34546 non-null  object 
 13  price            34546 non-null  int64  
 14  street_address   34542 non-null  object 
 15  TARGET           34546 non-null  object 
dtypes: float64(3), int64(4), object(9)
memory usage: 4.2+ MB


In [10]:
train.isna().sum()

Id                    0
bathrooms             0
bedrooms              0
building_id           0
created               0
description        1037
display_address      88
features              0
latitude              0
listing_id            0
longitude             0
manager_id            0
photos                0
price                 0
street_address        4
TARGET                0
dtype: int64

In [11]:
train = train.drop(columns=['description', 'display_address', 'street_address'])

In [12]:
train.head()

,Id,bathrooms,bedrooms,building_id,created,features,latitude,listing_id,longitude,manager_id,photos,price,TARGET
0,57094,1.0,3,0,2016-05-19 18:06:27,"['Laundry In Unit', 'No Fee', 'Elevator']",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,medium
1,33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,"['Dogs Allowed', 'Cats Allowed', 'No Fee']",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,medium
2,60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,"['Elevator', 'Hardwood Floors']",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,low
3,53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,"['Swimming Pool', 'Doorman', 'Elevator', 'Fitn...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,low
4,592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,"['Laundry in Building', 'Laundry in Unit', 'Di...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,low


In [13]:
train['created'] = pd.to_datetime(train['created'], errors='coerce')

In [14]:
train['day'] = train['created'].dt.day
train['month'] = train['created'].dt.month

In [15]:
train = train.drop(columns=['created'])

In [16]:
train.head()

,Id,bathrooms,bedrooms,building_id,features,latitude,listing_id,longitude,manager_id,photos,price,TARGET,day,month
0,57094,1.0,3,0,"['Laundry In Unit', 'No Fee', 'Elevator']",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,medium,19,5
1,33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,"['Dogs Allowed', 'Cats Allowed', 'No Fee']",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,medium,16,6
2,60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,"['Elevator', 'Hardwood Floors']",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,low,4,5
3,53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,"['Swimming Pool', 'Doorman', 'Elevator', 'Fitn...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,low,12,5
4,592,1.0,3,fee4d465932160318364d9d48d272879,"['Laundry in Building', 'Laundry in Unit', 'Di...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,low,16,6


**Добавим несколько признаков из features**

In [18]:
feat = set()
for i in train['features']:
  i = i[1:-1]
  if i.find(' * '):
    i = i.split(' * ')
  else:
    i = i.split(', ')
  feat.update(set(i))
print(feat)

{'', "'No Fee', 'Short Term Allowed', 'Gas and Electric Included in Rent'", "'Fitness Center', 'Outdoor Pool', 'Residents Lounge', 'WiFi Access', 'On-site Garage', 'On-site Parking Lot', 'Doorman', 'Concierge', 'No Fee', 'Exclusive', 'Elevator'", "'Elevator', 'Pre-War', 'Laundry in Unit', 'Dishwasher'", "'Doorman', 'Elevator', 'Fitness Center', 'Furnished', 'Loft', 'Laundry in Building', 'Laundry in Unit', 'High Speed Internet', 'Hardwood Floors', 'New Construction'", "'New Construction'", "'Common Outdoor Space', 'Cats Allowed', 'Dogs Allowed', 'Doorman', 'Elevator', 'Fitness Center', 'Laundry In Building', 'Bike Room', 'Concierge', 'Laundry in Building', 'Live-in Super'", "'Washer in Unit', 'Dryer in Unit', 'Exclusive'", "'Swimming Pool', 'Doorman', 'Elevator', 'Fitness Center', 'Pre-War', 'Laundry in Building', 'High Speed Internet'", "'Doorman', 'Elevator', 'Pool', 'Gym In Building', 'Roof Deck'", 'CHEF;S KITCHEN', "'Balcony', 'Doorman', 'Elevator', 'Garden/Patio', 'Terrace', 'Laun

In [19]:
#Лифт
def f_el(f):
  if 'Elevator' in f:
    return 1
  else:
    return 0

#Бассеин
def f_sp(f):
  if 'Swimming Pool' in f:
    return 1
  else:
    return 0

#Животные
def f_pets(f):
  if 'Cats' in f or 'Dogs' in f:
    return 1
  else:
    return 0

#Фитнес центр
def f_f(f):
  if 'Fitness' in f:
    return 1
  else:
    return 0

#Швейцар
def f_d(f):
  if 'Doorman' in f:
    return 1
  else:
    return 0

#Wifi
def f_wf(f):
  if 'WiFi' in f:
    return 1
  else:
    return 0

#Оборудование(техника)
def f_e(f):
  if 'Dishwasher' in f or 'tv' in f or 'microwave' in f or 'fireplace' in f or 'washer' in f:
    return 1
  else:
    return 0

#Парковка
def f_p(f):
  if 'parking' in f or 'Parking' in f or 'Garage' in f or 'garage' in f:
    return 1
  else:
    return 0

#Кухня
def f_k(f):
  if 'Kitchen' in f or 'kitchen' in f or 'KITCHEN' in f:
    return 1
  else:
    return 0

#Что-то королевское
def f_king(f):
  if 'King' in f or 'Queen' in f:
    return 1
  else:
    return 0

#Детская комната
def f_ch(f):
  if 'playroom' in f or 'children' in f or 'Children' in f:
    return 1
  else:
    return 0

#Места для хранения
def f_st(f):
  if 'storage' in f or 'closet' in f:
    return 1
  else:
    return 0

#Цетр города
def f_mid(f):
  if 'MIDTOWN' in f or 'midtown' in f:
    return 1
  else:
    return 0

#Хоршоий вид
def f_v(f):
  if 'view' in f or 'View' in f or 'VIEW' in f:
    return 1
  else:
    return 0

#Парк или огород
def f_p(f):
  if 'park' in f or 'garden' in f:
    return 1
  else:
    return 0

#Довоенный
def f_pw(f):
  if 'pre-war' in f:
    return 1
  else:
    return 0

In [20]:
train['Elevator'] = train['features'].apply(f_el)
train['Swimming Pool'] = train['features'].apply(f_sp)
train['Pets Allowed'] = train['features'].apply(f_pets)
train['Fitness Center'] = train['features'].apply(f_f)
train['Doorman'] = train['features'].apply(f_d)
train['WiFi'] = train['features'].apply(f_wf)
train['Equipment'] = train['features'].apply(f_e)
train['Parking'] = train['features'].apply(f_p)
train['Kitchen'] = train['features'].apply(f_k)
train['King or Queen'] = train['features'].apply(f_king)
train['Playroom'] = train['features'].apply(f_ch)
train['Storage'] = train['features'].apply(f_st)
train['Midtown'] = train['features'].apply(f_mid)
train['View'] = train['features'].apply(f_v)
train['Park and garden'] = train['features'].apply(f_p)
train['Pre-war'] = train['features'].apply(f_pw)

In [21]:
train.head()

,Id,bathrooms,bedrooms,building_id,features,latitude,listing_id,longitude,manager_id,photos,price,TARGET,day,month,Elevator,Swimming Pool,Pets Allowed,Fitness Center,Doorman,WiFi,Equipment,Parking,Kitchen,King or Queen,Playroom,Storage,Midtown,View,Park and garden,Pre-war
0,57094,1.0,3,0,"['Laundry In Unit', 'No Fee', 'Elevator']",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,medium,19,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,"['Dogs Allowed', 'Cats Allowed', 'No Fee']",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,medium,16,6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,"['Elevator', 'Hardwood Floors']",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,low,4,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,"['Swimming Pool', 'Doorman', 'Elevator', 'Fitn...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,low,12,5,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0
4,592,1.0,3,fee4d465932160318364d9d48d272879,"['Laundry in Building', 'Laundry in Unit', 'Di...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,low,16,6,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


**Количество комнат**

In [22]:
train['Rooms'] = train['bathrooms'] + train['bedrooms'] + train['Kitchen'] + train['Playroom']

**Количество фото**

In [23]:
def f_per(p):
  p = p[1:-1]
  p = p.split(', ')
  return len(p)

In [24]:
train['Number of photos'] = train['photos'].apply(f_per)

In [25]:
train.head()

,Id,bathrooms,bedrooms,building_id,features,latitude,listing_id,longitude,manager_id,photos,price,TARGET,day,month,Elevator,Swimming Pool,Pets Allowed,Fitness Center,Doorman,WiFi,Equipment,Parking,Kitchen,King or Queen,Playroom,Storage,Midtown,View,Park and garden,Pre-war,Rooms,Number of photos
0,57094,1.0,3,0,"['Laundry In Unit', 'No Fee', 'Elevator']",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,medium,19,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0,3
1,33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,"['Dogs Allowed', 'Cats Allowed', 'No Fee']",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,medium,16,6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,13
2,60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,"['Elevator', 'Hardwood Floors']",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,low,4,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,6
3,53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,"['Swimming Pool', 'Doorman', 'Elevator', 'Fitn...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,low,12,5,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,3.0,4
4,592,1.0,3,fee4d465932160318364d9d48d272879,"['Laundry in Building', 'Laundry in Unit', 'Di...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,low,16,6,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,4.0,3


In [26]:
train.shape

(34546, 32)

**Цена за комнату**

In [27]:
train['price_per_room'] = train['price']//train['Rooms']

In [28]:
train.head()

,Id,bathrooms,bedrooms,building_id,features,latitude,listing_id,longitude,manager_id,photos,price,TARGET,day,month,Elevator,Swimming Pool,Pets Allowed,Fitness Center,Doorman,WiFi,Equipment,Parking,Kitchen,King or Queen,Playroom,Storage,Midtown,View,Park and garden,Pre-war,Rooms,Number of photos,price_per_room
0,57094,1.0,3,0,"['Laundry In Unit', 'No Fee', 'Elevator']",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,medium,19,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0,3,1123.0
1,33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,"['Dogs Allowed', 'Cats Allowed', 'No Fee']",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,medium,16,6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,13,1285.0
2,60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,"['Elevator', 'Hardwood Floors']",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,low,4,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,6,1795.0
3,53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,"['Swimming Pool', 'Doorman', 'Elevator', 'Fitn...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,low,12,5,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,3.0,4,1133.0
4,592,1.0,3,fee4d465932160318364d9d48d272879,"['Laundry in Building', 'Laundry in Unit', 'Di...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,low,16,6,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,4.0,3,1423.0


**Добавим евклидово расстояние от дома до центра в км**

In [29]:
train_test = train.drop(columns=['Id', 'bathrooms', 'bedrooms', 'building_id', 'features', 'listing_id', 'manager_id', 'photos', 'price', 'TARGET', 'day', 'month', 'Elevator', 'Swimming Pool', 'Pets Allowed', 'Fitness Center', 'Doorman', 'WiFi', 'Equipment', 'Parking', 'Kitchen', 'King or Queen', 'Playroom', 'Storage', 'Midtown', 'View', 'Park and garden', 'Pre-war', 'Rooms', 'Number of photos', 'price_per_room'])

In [30]:
train_test.head()

,latitude,longitude
0,40.7647,-73.9918
1,40.7275,-74.0322
2,40.7798,-73.9751
3,40.7081,-74.0065
4,40.7416,-74.0025


In [31]:
def ll(f):
  p_1 = (f['latitude'], f['longitude'])
  p_2 = (40.75773000, -73.98570800)
  return great_circle(p_1, p_2).kilometers

In [32]:
train['Distance_km'] = train_test.apply(ll, axis = 1)

In [33]:
train.head()

,Id,bathrooms,bedrooms,building_id,features,latitude,listing_id,longitude,manager_id,photos,price,TARGET,day,month,Elevator,Swimming Pool,Pets Allowed,Fitness Center,Doorman,WiFi,Equipment,Parking,Kitchen,King or Queen,Playroom,Storage,Midtown,View,Park and garden,Pre-war,Rooms,Number of photos,price_per_room,Distance_km
0,57094,1.0,3,0,"['Laundry In Unit', 'No Fee', 'Elevator']",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,medium,19,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0,3,1123.0,0.929479
1,33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,"['Dogs Allowed', 'Cats Allowed', 'No Fee']",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,medium,16,6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,13,1285.0,5.161449
2,60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,"['Elevator', 'Hardwood Floors']",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,low,4,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,6,1795.0,2.611617
3,53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,"['Swimming Pool', 'Doorman', 'Elevator', 'Fitn...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,low,12,5,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,3.0,4,1133.0,5.790016
4,592,1.0,3,fee4d465932160318364d9d48d272879,"['Laundry in Building', 'Laundry in Unit', 'Di...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,low,16,6,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,4.0,3,1423.0,2.284249


In [34]:
train = train.drop(columns=['building_id', 'features', 'manager_id', 'Id', 'photos'])

In [35]:
train.head()

,bathrooms,bedrooms,latitude,listing_id,longitude,price,TARGET,day,month,Elevator,Swimming Pool,Pets Allowed,Fitness Center,Doorman,WiFi,Equipment,Parking,Kitchen,King or Queen,Playroom,Storage,Midtown,View,Park and garden,Pre-war,Rooms,Number of photos,price_per_room,Distance_km
0,1.0,3,40.7647,7039994,-73.9918,4495,medium,19,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0,3,1123.0,0.929479
1,1.0,1,40.7275,7166774,-74.0322,2570,medium,16,6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,13,1285.0,5.161449
2,1.0,0,40.7798,6962716,-73.9751,1795,low,4,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,6,1795.0,2.611617
3,1.0,2,40.7081,7002458,-74.0065,3400,low,12,5,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,3.0,4,1133.0,5.790016
4,1.0,3,40.7416,7170465,-74.0025,5695,low,16,6,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,4.0,3,1423.0,2.284249


In [36]:
train.shape

(34546, 29)

In [37]:
train.isna().sum()

bathrooms           0
bedrooms            0
latitude            0
listing_id          0
longitude           0
price               0
TARGET              0
day                 0
month               0
Elevator            0
Swimming Pool       0
Pets Allowed        0
Fitness Center      0
Doorman             0
WiFi                0
Equipment           0
Parking             0
Kitchen             0
King or Queen       0
Playroom            0
Storage             0
Midtown             0
View                0
Park and garden     0
Pre-war             0
Rooms               0
Number of photos    0
price_per_room      0
Distance_km         0
dtype: int64

**Делаем то же самое с test**

In [38]:
test.head()

,Id,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address
0,low,2.0,3,87c4e08083ac83618dd9fdbf849331fe,2016-06-21 03:21:59,Rent listed is net effective after one month f...,E 77 St.,"['Pre-War', 'Laundry in Unit', 'Dishwasher', '...",40.7716,7189305,-73.9544,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7189305_f47ed5c...,4675,339 E 77 St.
1,low,1.0,2,850c3d4412aebb3335273ed95f3e4bd4,2016-06-11 03:47:06,A FABULOUS 2BR IN THE UPPER EAST SIDE! PERFECT...,E 112 Street,[],40.7944,7140670,-73.9401,3e9f0fa34d67e5d61aae56776ecb8cee,['https://photos.renthop.com/2/7140670_6c5824f...,2100,252 E 112 Street
2,low,1.0,2,84ddf917a091828ab9baedc0c926470e,2016-04-17 02:45:19,This is a great true 2 bedroom in a prime Midt...,Ninth Avenue,"['Dogs Allowed', 'Cats Allowed']",40.7650,6886239,-73.9882,10b1ae0a38d50b7ba0cee612d14af9eb,['https://photos.renthop.com/2/6886239_533bbee...,2800,783 Ninth Avenue
3,high,2.0,3,315a2f868a195b076e5be73411110da2,2016-05-07 11:44:35,Brand new custom renovation in beautiful three...,East 12th street,"['Laundry In Unit', 'No Fee', 'Washer/Dryer in...",40.7288,6981146,-73.9808,e9920062e07ee893c10e38d0259665b0,['https://photos.renthop.com/2/6981146_f4a7400...,5500,186 Avenue B
4,medium,1.0,3,9b525bacc8d294728c35f27c04e22f46,2016-06-24 07:33:26,"Welcome Home! Located on a Prime UWS Block, Ba...",West 86th&Columbus,"['Private Outdoor Space', 'Elevator', 'Laundry...",40.7864,7210864,-73.9713,6d389fbe372d4d30b17733caa9370f95,['https://photos.renthop.com/2/7210864_e8528f4...,3995,47 West 86th street


In [39]:
test = test.drop(columns=['description', 'display_address', 'street_address'])

In [40]:
test['created'] = pd.to_datetime(test['created'], errors='coerce')

In [41]:
test['day'] = test['created'].dt.day
test['month'] = test['created'].dt.month

In [42]:
test = test.drop(columns=['created'])

In [43]:
test['Elevator'] = test['features'].apply(f_el)
test['Swimming Pool'] = test['features'].apply(f_sp)
test['Pets Allowed'] = test['features'].apply(f_pets)
test['Fitness Center'] = test['features'].apply(f_f)
test['Doorman'] = test['features'].apply(f_d)
test['WiFi'] = test['features'].apply(f_wf)
test['Equipment'] = test['features'].apply(f_e)
test['Parking'] = test['features'].apply(f_p)
test['Kitchen'] = test['features'].apply(f_k)
test['King or Queen'] = test['features'].apply(f_king)
test['Playroom'] = test['features'].apply(f_ch)
test['Storage'] = test['features'].apply(f_st)
test['Midtown'] = test['features'].apply(f_mid)
test['View'] = test['features'].apply(f_v)
test['Park and garden'] = test['features'].apply(f_p)
test['Pre-war'] = test['features'].apply(f_pw)

In [44]:
test['Rooms'] = test['bathrooms'] + test['bedrooms'] + test['Kitchen'] + test['Playroom']

In [45]:
test['Number of photos'] = test['photos'].apply(f_per)

In [46]:
test['price_per_room'] = test['price']//test['Rooms']

In [47]:
test_test = test.drop(columns=['Id', 'bathrooms', 'bedrooms', 'building_id', 'features', 'listing_id', 'manager_id', 'photos', 'price', 'day', 'month', 'Elevator', 'Swimming Pool', 'Pets Allowed', 'Fitness Center', 'Doorman', 'WiFi', 'Equipment', 'Parking', 'Kitchen', 'King or Queen', 'Playroom', 'Storage', 'Midtown', 'View', 'Park and garden', 'Pre-war', 'Rooms', 'Number of photos', 'price_per_room'])

In [48]:
test['Distance_km'] = test_test.apply(ll, axis = 1)

In [49]:
test = test.drop(columns=['building_id', 'features', 'manager_id', 'photos'])

In [50]:
test.head()

,Id,bathrooms,bedrooms,latitude,listing_id,longitude,price,day,month,Elevator,Swimming Pool,Pets Allowed,Fitness Center,Doorman,WiFi,Equipment,Parking,Kitchen,King or Queen,Playroom,Storage,Midtown,View,Park and garden,Pre-war,Rooms,Number of photos,price_per_room,Distance_km
0,low,2.0,3,40.7716,7189305,-73.9544,4675,21,6,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,5.0,8,935.0,3.054658
1,low,1.0,2,40.7944,7140670,-73.9401,2100,11,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,4,700.0,5.601326
2,low,1.0,2,40.7650,6886239,-73.9882,2800,17,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,7,933.0,0.835190
3,high,2.0,3,40.7288,6981146,-73.9808,5500,7,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.0,7,1100.0,3.243338
4,medium,1.0,3,40.7864,7210864,-73.9713,3995,24,6,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,4.0,10,998.0,3.411038


In [51]:
test.shape

(14806, 29)

In [52]:
test.isna().sum()

Id                  0
bathrooms           0
bedrooms            0
latitude            0
listing_id          0
longitude           0
price               0
day                 0
month               0
Elevator            0
Swimming Pool       0
Pets Allowed        0
Fitness Center      0
Doorman             0
WiFi                0
Equipment           0
Parking             0
Kitchen             0
King or Queen       0
Playroom            0
Storage             0
Midtown             0
View                0
Park and garden     0
Pre-war             0
Rooms               0
Number of photos    0
price_per_room      0
Distance_km         0
dtype: int64

**Удаляем price_per_room потому что там есть бесконечные значения**

In [53]:
test = test.drop(columns=['price_per_room'])

In [54]:
train = train.drop(columns=['price_per_room'])

In [55]:
X_train = train.drop('TARGET', axis = 1)
y_train = train['TARGET']
X_test = test.drop('Id', axis = 1)
y_test = test['Id']

# **Модель**

In [56]:
print(np.any(np.isnan(X_train)))
print(np.all(np.isfinite(X_train)))

False
True


In [57]:
print(np.any(np.isnan(X_test)))
print(np.all(np.isfinite(X_test)))

False
True


In [58]:
log_reg = LogisticRegression(multi_class='ovr', solver='lbfgs', class_weight={'low':0.33, 'high':2.9, 'medium':1.})
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average='micro'))

0.4436039443468864
0.4436039443468864


In [60]:
s = pd.DataFrame.from_dict({'Id':test['Id'], 'TARGET': y_pred})
s.head()

,Id,TARGET
0,low,low
1,low,high
2,low,high
3,high,low
4,medium,low
